# Download the dataset

### Dataset

In this homework, we'll build a model for predicting if we have an image of a bee or a wasp. 
For this, we will use the "Bee or Wasp?" dataset that was obtained from [Kaggle](https://www.kaggle.com/datasets/jerzydziewierz/bee-vs-wasp) and slightly rebuilt. 

You can download the dataset for this homework from [here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip):

```bash
wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
unzip data.zip
```

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch. 

> **Note:** you will need an environment with a GPU for this homework. We recommend to use [Saturn Cloud](https://bit.ly/saturn-mlzoomcamp). 
> You can also use a computer without a GPU (e.g. your laptop), but it will be slower.


### Data Preparation

The dataset contains around 2500 images of bees and around 2100 images of wasps. 

The dataset contains separate folders for training and test sets. 


In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip


In [2]:
!unzip data.zip
!rm data.zip

unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.
rm: cannot remove 'data.zip': No such file or directory


# Imports

In [14]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

AUTOTUNE = tf.data.AUTOTUNE

# Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`


In [2]:
def make_model(size=150):
    
    inputs = keras.Input(shape=(size, size, 3))
    # when using image_dataset_from_directory we need to include rescaling layer
    rescaled = layers.Rescaling(1./255)(inputs)
    conv2d = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(rescaled)
    pooled = layers.MaxPooling2D()(conv2d)
    flattened = layers.Flatten()(pooled)
    
    fully_connected = layers.Dense(units=64, activation='relu')(flattened)
    
    outputs = layers.Dense(1, activation=keras.activations.sigmoid)(fully_connected)
    
    model = keras.Model(inputs, outputs)
    
    #########################################

    optimizer = keras.optimizers.SGD(lr=0.002, momentum=0.8)
    loss = keras.losses.BinaryCrossentropy()

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

In [3]:
model = make_model()

2023-11-21 21:20:55.513419: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-21 21:20:55.520244: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-21 21:20:55.520901: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-21 21:20:55.521863: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

* `mean squared error`
* `binary crossentropy`
* `categorical crossentropy`
* `cosine similarity`

### Question 2

What's the number of parameters in the convolutional layer of our model? You can use the `summary` method for that. 

* 1 
* 65
* 896
* 11214912


In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 150, 150, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                             

### Generators and Training

For the next two questions, use the following data generator for both train and test sets:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/test directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets. 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [19]:
def read_ds():
    train_ds = tf.keras.utils.image_dataset_from_directory(
        'data/train/',
        # The default one is 32 so set to None since we will batch later
        batch_size=None,
        image_size=(150, 150),
        label_mode='binary',
    )

    val_ds = tf.keras.utils.image_dataset_from_directory(
        'data/test/',
        batch_size=None,
        image_size=(150, 150),
        label_mode='binary',
    )
    return train_ds, val_ds

In [6]:
def configure_for_performance(ds, batch_size=20, data_aug_fn=None):
    ds = ds.cache()
    if data_aug_fn:
        ds = ds.map(data_aug_fn, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [20]:
train_ds, val_ds = read_ds()
train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

Found 3681 files belonging to 2 classes.
Found 918 files belonging to 2 classes.


In [10]:
history = model.fit(train_ds, 
                    epochs=10, 
                    validation_data=val_ds,
                   )

Epoch 1/10


2023-11-21 21:25:53.689979: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-11-21 21:25:54.395173: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-21 21:25:54.395684: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-21 21:25:54.395724: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-11-21 21:25:54.396244: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-21 21:25:54.396336: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


185/185 [==============================] - 7s 19ms/step - loss: 0.6522 - accuracy: 0.6061 - val_loss: 0.6043 - val_accuracy: 0.6242
Epoch 2/10
185/185 [==============================] - 2s 13ms/step - loss: 0.5753 - accuracy: 0.6971 - val_loss: 0.5455 - val_accuracy: 0.7331
Epoch 3/10
185/185 [==============================] - 2s 13ms/step - loss: 0.5325 - accuracy: 0.7419 - val_loss: 0.5397 - val_accuracy: 0.7266
Epoch 4/10
185/185 [==============================] - 2s 13ms/step - loss: 0.5087 - accuracy: 0.7637 - val_loss: 0.5325 - val_accuracy: 0.7309
Epoch 5/10
185/185 [==============================] - 2s 13ms/step - loss: 0.4900 - accuracy: 0.7770 - val_loss: 0.5186 - val_accuracy: 0.7560
Epoch 6/10
185/185 [==============================] - 2s 13ms/step - loss: 0.4630 - accuracy: 0.7908 - val_loss: 0.5129 - val_accuracy: 0.7603
Epoch 7/10
185/185 [==============================] - 2s 12ms/step - loss: 0.4334 - accuracy: 0.8052 - val_loss: 0.5065 - val_accuracy: 0.7647
Epoch 8/10

### Question 3

What is the median of training accuracy for all the epochs for this model?

* 0.20
* 0.40
* 0.60
* 0.80

In [15]:
np.median(history.history['accuracy'])

0.7838902473449707

### Question 4

What is the standard deviation of training loss for all the epochs for this model?

* 0.031
* 0.061
* 0.091
* 0.131

In [17]:
np.std(history.history['loss'])

0.09140642920399505

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=50,`
* `width_shift_range=0.1,`
* `height_shift_range=0.1,`
* `zoom_range=0.1,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`


In [24]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(factor=0.28, fill_mode='nearest'),
    layers.RandomZoom(height_factor=0.1, fill_mode='nearest'),
    layers.RandomWidth(factor=0.1, interpolation='nearest'),
    layers.RandomHeight(factor=0.1, interpolation='nearest'),
    layers.Resizing(150, 150)
])

In [25]:
train_ds, val_ds = read_ds()
train_ds = configure_for_performance(train_ds, data_aug_fn=lambda img, label: (data_augmentation(img), label))
val_ds = configure_for_performance(val_ds)

Found 3681 files belonging to 2 classes.
Found 918 files belonging to 2 classes.


### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
> **Note:** make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.18
* 0.48
* 0.78
* 0.108

In [26]:
history = model.fit(train_ds, 
                    epochs=10, 
                    validation_data=val_ds,
                   )

Epoch 1/10
185/185 [==============================] - 11s 56ms/step - loss: 0.4976 - accuracy: 0.7702 - val_loss: 0.5755 - val_accuracy: 0.7320
Epoch 2/10
185/185 [==============================] - 9s 49ms/step - loss: 0.5053 - accuracy: 0.7607 - val_loss: 0.4830 - val_accuracy: 0.7712
Epoch 3/10
185/185 [==============================] - 9s 49ms/step - loss: 0.4891 - accuracy: 0.7753 - val_loss: 0.5067 - val_accuracy: 0.7669
Epoch 4/10
185/185 [==============================] - 9s 48ms/step - loss: 0.4887 - accuracy: 0.7732 - val_loss: 0.5226 - val_accuracy: 0.7516
Epoch 5/10
185/185 [==============================] - 9s 49ms/step - loss: 0.5063 - accuracy: 0.7577 - val_loss: 0.4863 - val_accuracy: 0.7691
Epoch 6/10
185/185 [==============================] - 9s 49ms/step - loss: 0.5041 - accuracy: 0.7675 - val_loss: 0.5202 - val_accuracy: 0.7571
Epoch 7/10
185/185 [==============================] - 9s 49ms/step - loss: 0.4886 - accuracy: 0.7767 - val_loss: 0.5005 - val_accuracy: 0.765

In [28]:
np.mean(history.history['val_loss'])

0.5268334418535232

### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

* 0.38
* 0.58
* 0.78
* 0.98

In [31]:
np.mean(history.history['val_accuracy'][-5:])

0.7457516431808472